In [1]:
# 4.1 Section Overview 
'''
This is one of the most important sections of the course.
Analytic Functions calculate an aggregate value based on a group of rows.
Unlike aggregate functions, however, Analytic Functions can return multiple rows for each group. 

We can use Analytic Functions to compute 
  - aggregated values
  - moving averages
  - running totals
  - percentages or top-N results within a group 
  and much, much more as you will see in this section.

We will cover in detail the syntax for Analytical Functions and have a series of code alongs, practice scenarios 
and assignment questions.By the end of this section you will have a solid understanding of Analytical Functions.
'''

'\nThis is one of the most important sections of the course.\nAnalytic Functions calculate an aggregate value based on a group of rows.\nUnlike aggregate functions, however, Analytic Functions can return multiple rows for each group. \n\nWe can use Analytic Functions to compute \n  - aggregated values\n  - moving averages\n  - running totals\n  - percentages or top-N results within a group \n  and much, much more as you will see in this section.\n\nWe will cover in detail the syntax for Analytical Functions and have a series of code alongs, practice scenarios and assignment questions.\nBy the end of this section you will have a solid understanding of Analytical Functions.\n'

In [4]:
# Introduction to Analytical Functions 
'''
 * Analytical Functions use the OVER() clause and are quite different to standard SQL.
 * Analytics Functions are also know as Window Functions 
 * Analyics Functions compute their result based on window of one or more rows.
 * Unlike aggregation operation such as Group by , they do not collapse rows.
'''
# Solution 1: explain 
'''
# Population Percent for each region , without using analytical functions
select * from eba_countries;

SELECT 
  a.name,
  a.region_id,
  a.population

FROM eba_countries a;

SELECT 
  a.name,
  a.region_id,
  a.population,
  b.region_pop,
  round(a.population / b.region_pop * 100, 2) as pct_of_region_total
FROM eba_countries a
LEFT JOIN
(select 
    region_id, sum(population) as region_pop
    FROM eba_countries
    GROUP by region_id) b
on a.region_id = b.region_id;

# Check explain for cost : 7 Here , and lot of steps 
'''

# Solution 2 with analytical functions
'''
select 
 name,
 region_id,
 population,
 sum(population) over(partition by region_id) -- it gives total population of region_id
FROM eba_countries;

select 
 name,
 region_id,
 population,
 round(population / sum(population) over(partition by region_id)* 100 , 2) as pct_of_region_app

FROM eba_countries;

# explain : cost is 4 
'''

# Partition BY mutlab , partition kar do , as we learn in spark like folder wise then distribute the data
####### SYNTAX ########## 
'''
 WINDOW FUNCTION ----------> SUM(Population)
 OVER(
    PARTITION BY -----> PARTIITION BY REGION_ID
    ORDER BY     -----> ORDER BY REGION_ID
    WINDOW_FRAME ------> ROWS UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
 )
'''

'\nselect \n name,\n region_id,\n population,\n sum(population) over(partition by region_id)\n\nFROM eba_countries;\n\nselect \n name,\n region_id,\n population,\n round(population / sum(population) over(partition by region_id)* 100 , 2) as pct_of_region_app\n\nFROM eba_countries;\n\n# explain : cost is 4 \n'

In [2]:
# Introduction to the Over Clause
'''
 WINDOW FUNCTION ----------> SUM(Population)
 OVER(
    PARTITION BY -----> PARTIITION BY REGION_ID
    ORDER BY     -----> ORDER BY REGION_ID
    WINDOW_FRAME ------> ROWS UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
 )
'''
# Example 
'''
-- if we dont specify the partition ,over will take in all the table
select  sum(population) from eba_countries; --7713500418

select name, sum(population) from eba_countries; --invalid as group by is required

select name, sum(population) over() from eba_countries; -- apply a window functions to every row in table

'''

In [1]:
# The Over Clause - Partition By
'''
 WINDOW FUNCTION ----------> SUM(Population)
 OVER(
    PARTITION BY -----> PARTIITION BY REGION_ID
                 -----> PARTIITION BY REGION_ID, SUB_REGION_ID

    ORDER BY     -----> ORDER BY REGION_ID
    WINDOW_FRAME ------> ROWS UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
 )
'''
# Example 
'''
-- caclualate the sum of population using region_id
SELECT 
 population,
 region_id,
 sub_region_id,
 sum(population) over(partition by region_id)
FROM eba_countries;

-- caclualate the sum of population using region_id, sub_region_id
SELECT 
 population,
 region_id,
 sub_region_id,
 sum(population) over(partition by region_id, sub_region_id)

FROM eba_countries;
'''

'\n-- caclualate the sum of population using region_id\nSELECT \n population,\n region_id,\n sub_region_id,\n sum(population) over(partition by region_id)\nFROM eba_countries;\n\n-- caclualate the sum of population using region_id, sub_region_id\nSELECT \n population,\n region_id,\n sub_region_id,\n sum(population) over(partition by region_id, sub_region_id)\n\nFROM eba_countries;\n'

In [3]:
# The Over Clause - Order By
'''
 WINDOW FUNCTION
 OVER(
    PARTITION BY ...
    ORDER BY     -----> ORDER BY POPULATION
                 ------> ORDER BY POPULATION, REGION_ID
    WINDOW_FRAME ..
 )
'''
# Example1
'''
SELECT 
 name,
 population,
 region_id,
 sum(population) over(order by region_id asc) -- over() clause ran on entrie table 

FROM eba_countries;
'''
# Example 2 
'''
SELECT 
 name,
 population,
 region_id,
 sub_region_id,
 sum(population) over(order by region_id, sub_region_id desc) -- entrie table 
 -- sum(population) over(order by region_id,sub_region_id asc) -- entrie table 

FROM eba_countries;
'''

# Example 3
'''
SELECT 
 name,
 population,
 region_id,
 sub_region_id,
 sum(population) over(partition by region_id order by sub_region_id desc)
FROM eba_countries;
'''

'\nSELECT \n name,\n population,\n region_id,\n sub_region_id,\n sum(population) over(partition by region_id order by sub_region_id desc)\nFROM eba_countries;\n'

In [4]:
# An Introduction To Windows Frames
# SYNTAX
'''
 WINDOW FUNCTION
 OVER(
    PARTITION BY ...
    ORDER BY     ...
    WINDOW_FRAME ----------> ROWS|RANGE bound
                 ----------> ROWS|RANGE BETWEEN lower_bound AND upper_bound
 )
'''

# 
'''
 1. 1. preceding means in rows means last row 
 2. 1. preceding means in range means one unit 
'''

'\n 1. 1. preceding means in rows means last row \n 2. 1. preceding means in range means one unit \n'

In [7]:
# Window Frame - Rows -> rows will check the preceding rows and following rows ..
'''
select
  name, 
  region_id,
  sub_region_id,
  population,
  sum(population) over(partition by region_id, sub_region_id order by sub_region_id) as defaul,
  sum(population) over(partition by region_id, sub_region_id order by sub_region_id rows between unbounded preceding and unbounded following) as a ,
  sum(population) over(partition by region_id, sub_region_id order by sub_region_id rows between 1 preceding and 1 following) as b ,
  sum(population) over(partition by region_id, sub_region_id order by sub_region_id rows between current row and 2 following) as c

from eba_countries;

'''

'\nselect\n  name, \n  region_id,\n  sub_region_id,\n  population,\n  sum(population) over(partition by region_id, sub_region_id order by sub_region_id) as defaul,\n  sum(population) over(partition by region_id, sub_region_id order by sub_region_id rows between unbounded preceding and unbounded following) as a ,\n  sum(population) over(partition by region_id, sub_region_id order by sub_region_id rows between 1 preceding and 1 following) as b ,\n  sum(population) over(partition by region_id, sub_region_id order by sub_region_id rows between current row and 2 following) as c\n\nfrom eba_countries;\n\n'

In [6]:
# Window Frame - Range [ range will check the range from preceding and following ]

'''
select
  name, 
  region_id,
  sub_region_id,
  population,
  sum(population) over(partition by region_id order by sub_region_id range between 1 preceding and 1 following) as a,
  sum(population) over(partition by region_id order by sub_region_id range between 0 preceding and 70 following) as b

from eba_countries;
'''

# For Example 
'''
 we have region_id [ 10,20,30,40,50 ]
 so we can define the range in difference of 10.
'''

# This is because we have defined sub_region_id in the Order By clause - so the values for this 
# field will be used as the range.

'\n we have region_id [ 10,20,30,40,50 ]\n so we can define the range in difference of 10.\n'

In [6]:
# Window Frame - Default Behaviour
'''
SELECT
  name, 
  region_id, 
  sub_region_id,
  population,
  sum(population) over(partition by region_id order by sub_region_id) as a,
  sum(population) over(partition by region_id) as b

FROM eba_countries;

 - IF ORDER BY is specified, then the frame is RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW 
 - IF ORDER BY is NOT  specified, then the frame is ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING 
'''

'\nSELECT\n  name, \n  region_id, \n  sub_region_id,\n  population,\n  sum(population) over(partition by region_id order by sub_region_id) as a,\n  sum(population) over(partition by region_id) as b\n\nFROM eba_countries;\n\n - IF ORDER BY is specified, then the frame is RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW \n - IF ORDER BY is NOT  specified, then the frame is ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING \n'

In [9]:
# Quick Note on Window Frames
'''
SELECT
  name, 
  region_id, 
  sub_region_id,
  population,
  sum(population) over(partition by region_id, sub_region_id order by sub_region_id rows 2 preceding) as a,
  sum(population) over(partition by region_id, sub_region_id order by sub_region_id rows between 2 preceding and current row) as b,
  -- sum(population) over(partition by region_id, sub_region_id order by sub_region_id rows 2 following) as a,
  sum(population) over(partition by region_id, sub_region_id order by sub_region_id rows between current row and 2 following) as c

FROM eba_countries;
'''
# Types of Window Functions
'''
 1. AGGREGATE
   - SUM()
   - AVG()
   - MAX()
   - MIN()
   - COUNT()

 2. RANKING
    - ROW_NUMBER()
    - RANK()
    - DENSE_RANK()
       * example top 10 , top 5 like that 

 3. DISTRIBUTION
   - PERCENT_RANK()
   - CUME_DIST()

 4. ANALYTICAL
    - LEAD()
    - LAG()
    - NTILE()
    - NTH_VALUE()
'''

'\n 1. AGGREGATE\n   - SUM()\n   - AVG()\n   - MAX()\n   - MIN()\n   - COUNT()\n\n 2. RANKING\n    - ROW_NUMBER()\n    - RANK()\n    - DENSE_RANK()\n\n 3. DISTRIBUTION\n   - PERCENT_RANK()\n   - CUME_DIST()\n\n 4. ANALYTICAL\n    - LEAD()\n    - LAG()\n    - NTILE()\n    - NTH_VALUE()\n'

In [10]:
# Aggregate Functions: SUM , AVG, MAX, MIN , COUNT
'''
SELECT
  name, 
  region_id, 
  sum(population) over(),
  max(population) over(),
  min(population) over(),
  count(population) over(),
  avg(population) over()

FROM eba_countries;


SELECT
  name, 
  region_id, 
  sum(population) over(partition by region_id),
  max(population) over(partition by region_id),
  min(population) over(partition by region_id),
  count(population) over(partition by region_id),
  avg(population) over(partition by region_id)

FROM eba_countries;
'''

'\nSELECT\n  name, \n  region_id, \n  sum(population) over(),\n  max(population) over(),\n  min(population) over(),\n  count(population) over(),\n  avg(population) over()\n\nFROM eba_countries;\n\n\nSELECT\n  name, \n  region_id, \n  sum(population) over(partition by region_id),\n  max(population) over(partition by region_id),\n  min(population) over(partition by region_id),\n  count(population) over(partition by region_id),\n  avg(population) over(partition by region_id)\n\nFROM eba_countries;\n'

In [11]:
# Analytical Functions: LAG, LEAD, NTILE, and NTH_VALUE
'''
-- LAG
SELECT
  name, 
  population, 
  lag(name) over(order by name) as a ,
  lag(name, 2) over(order by name) as b,
  lag(name, 2, 'blank') over(order by name) as c

FROM eba_countries;

-- LEAD
SELECT
  name, 
  population, 
  lead(name) over(order by name) as a ,
  lead(name, 2) over(order by name) as b,
  lead(name, 2, 'blank') over(order by name) as c

FROM eba_countries;

-- NTILE
SELECT
  name, 
  region_id,
  population, 
  ntile(3) over(order by population) as a,
  ntile(3) over(partition by region_id  order by population) as b

FROM eba_countries;

-- NTH_VALUE()
SELECT
  name, 
  region_id,
  population, 
  nth_value(name, 2) over() as a ,
  nth_value(name, 2) over(partition by region_id)

FROM eba_countries;

'''

"\n-- LAG\nSELECT\n  name, \n  population, \n  lag(name) over(order by name) as a ,\n  lag(name, 2) over(order by name) as b,\n  lag(name, 2, 'blank') over(order by name) as c\n\nFROM eba_countries;\n\n-- LEAD\nSELECT\n  name, \n  population, \n  lead(name) over(order by name) as a ,\n  lead(name, 2) over(order by name) as b,\n  lead(name, 2, 'blank') over(order by name) as c\n\nFROM eba_countries;\n\n-- NTILE\nSELECT\n  name, \n  region_id,\n  population, \n  ntile(3) over(order by population) as a,\n  ntile(3) over(partition by region_id  order by population) as b\n\nFROM eba_countries;\n\n-- NTH_VALUE()\nSELECT\n  name, \n  region_id,\n  population, \n  nth_value(name, 2) over() as a ,\n  nth_value(name, 2) over(partition by region_id)\n\nFROM eba_countries;\n\n"

In [12]:
# Ranking Functions: ROW_NUMBER, RANK, DENSE_RANK
'''
SELECT
  name, 
  population,
  row_number() over(order by population),
  rank() over(order by population),
  dense_rank() over(order by population)

FROM eba_countries;

-- For Tie RANK & dense_rank gives same number 
-- But for next row ,rank() skip the number and gives next but  dense_rank() doesnot skips gives next subsequent number 

SELECT
  name, 
  region_id,
  population,
  -- row_number() over(partition by region_id), --- error in oracle
  row_number() over(partition by region_id order by population),
  rank() over(partition by region_id order by population),
  dense_rank() over( partition by region_id order by population)

FROM eba_countries;
'''

In [13]:
# DISTRIBUTION Functions: PRECENT_RANK and CUME_DIST(Cumulative Distribution)
'''
-- NTH_VALUE()
SELECT
  name, 
  population,
  percent_rank() over(order by population), 
  cume_dist() over(order by population)

FROM eba_countries;


SELECT
  name, 
  region_id,
  population,
  percent_rank() over(partition by region_id order by population), 
  cume_dist() over(partition by region_id order by population)

FROM eba_countries;


'''

In [1]:
# Order Of Execution
'''
 - FROM 
 - JOIN 
 - WHERE  => Row Level Filter
 - GROUP BY 
 - HAVING 
 - WINDOW **
 - SELECT
 - ORDER BY 
 - LIMIT / FETCH / TOP
'''

'\n - FROM \n - JOIN \n - WHERE  => Row Level Filter\n - GROUP BY \n - HAVING \n - WINDOW **\n - SELECT\n - ORDER BY \n - LIMIT / FETCH / TOP\n'

In [2]:
# Practise Scenario Walkthrough 1
'''
-- Find the population difference between the countries in the current row vs the country with the highest population

select 
  name,
  population,
  max(population) over(),
  population - max(population) over() as diff_from_max_pop

FROM eba_countries;
'''

'\n-- Find the population difference between the countries in the current row vs the country with the highest population\n\nselect \n  name,\n  population,\n  max(population) over(),\n  population - max(population) over() as diff_from_max_pop\n\nFROM eba_countries;\n'

In [3]:
# Practise Scenario Walkthrough 2
'''
-- Arrange the countries in order of population from largetst to smallest and find the difference between the population of the 
-- current rows country and country immediately following it 


select 
  name,
  population,
  lead(population) over(order by population desc) as immediate_row,
  population - lead(population, 1) over(order by population desc) as variance

FROM eba_countries
--order by population desc;
'''

'\n-- Arrange the countries in order of population from largetst to smallest and find the difference between the population of the current rows country and country immediately following it \n\n\nselect \n  name,\n  population,\n  lead(population) over(order by population desc) as immediate_row,\n  population - lead(population, 1) over(order by population desc) as variance\n\nFROM eba_countries\n--order by population desc;\n'

In [4]:
# Practise Scenario Walkthrough 3
'''
-- Flag all countries that are in the top 10 percentile for their region in terms of population [ Higesht to lowest]
 
select * from (
    select 
    name,
    region_id,
    population,
    percent_rank() over(partition by region_id order by population) pop_percentile1,
    round(percent_rank() over(partition by region_id order by population) * 100, 2) as pop_percentile

    FROM eba_countries
) where pop_percentile > 10;

-- 10% means 0.1
    select 
    name,
    region_id,
    population,
    percent_rank() over(partition by region_id order by population) pop_percentile1,
    case when 
           percent_rank() over(partition by region_id order by population desc) <=0.1 then 1
         else 0 
    end as top10_perecentile

    FROM eba_countries;
'''

'\n-- Flag all countries that are in the top 10 percentile for their region in terms of population [ Higesht to lowest]\n \nselect * from (\n    select \n    name,\n    region_id,\n    population,\n    percent_rank() over(partition by region_id order by population) pop_percentile1,\n    round(percent_rank() over(partition by region_id order by population) * 100, 2) as pop_percentile\n\n    FROM eba_countries\n) where pop_percentile > 10;\n\n-- 10% means 0.1\n    select \n    name,\n    region_id,\n    population,\n    percent_rank() over(partition by region_id order by population) pop_percentile1,\n    case when \n           percent_rank() over(partition by region_id order by population desc) <=0.1 then 1\n         else 0 \n    end as top10_perecentile\n\n    FROM eba_countries;\n'

In [6]:
# Practise Scenario Walkthrough 4
'''
-- Order the countries by population in ascending order and create a running total field for the population
 
 select 
    name,
    population,
    sum(population) over(order by population asc rows between unbounded preceding and current row) as running_total_population,
    sum(population) over(order by population asc rows unbounded preceding) as running_total_population1

 FROM eba_countries


'''

'\n-- Order the countries by population in ascending order and create a running total field for the population\n \n select \n    name,\n    population,\n    sum(population) over(order by population asc rows between unbounded preceding and current row) as running_total_population,\n    sum(population) over(order by population asc rows unbounded preceding) as running_total_population1\n\n FROM eba_countries\n\n\n'

In [7]:
# Assigment Data Prepration
'''
-- Customer & Order data models
select * from orders;
select * from order_items;
 -- order_id, line_item_id, product_id, unit_price, quantity

select count(*) from order_items; --3914
select count(distinct order_id) from order_items; --1950


CREATE VIEW V_ORDERS AS

(SELECT A.*, B.ORDER_TOTAL

FROM ORDERS A

LEFT JOIN

(SELECT ORDER_ID, SUM(UNIT_PRICE * QUANTITY) AS ORDER_TOTAL FROM ORDER_ITEMS GROUP BY ORDER_ID) B

ON A.ORDER_ID = B.ORDER_ID

WHERE A.ORDER_STATUS = 'COMPLETE');


'''

"\n-- Customer & Order data models\nselect * from orders;\nselect * from order_items;\n -- order_id, line_item_id, product_id, unit_price, quantity\n\nselect count(*) from order_items; --3914\nselect count(distinct order_id) from order_items; --1950\n\n\nCREATE VIEW V_ORDERS AS\n\n(SELECT A.*, B.ORDER_TOTAL\n\nFROM ORDERS A\n\nLEFT JOIN\n\n(SELECT ORDER_ID, SUM(UNIT_PRICE * QUANTITY) AS ORDER_TOTAL FROM ORDER_ITEMS GROUP BY ORDER_ID) B\n\nON A.ORDER_ID = B.ORDER_ID\n\nWHERE A.ORDER_STATUS = 'COMPLETE');\n\n\n"

In [8]:
#Assignment Question 1
'''
For each customer rank their orders from highest to lowest in terms of order total using Analytical Functions (use RANK())
(For the assignment questions we are using V_ORDERS created in the "Assignment Data Preparation" lecture)


select * from v_orders;

select 
  order_id,
  customer_id,
  order_total,
  RANK() OVER(partition by customer_id order by order_total desc) as order_total_rank  

FROM v_orders

'''

'\nFor each customer rank their orders from highest to lowest in terms of order total using Analytical Functions (use RANK())\n(For the assignment questions we are using V_ORDERS created in the "Assignment Data Preparation" lecture)\n\n\nselect * from v_orders;\n\nselect \n  order_id,\n  customer_id,\n  order_total,\n  RANK() OVER(partition by customer_id order by order_total desc) as order_total_rank  \n\nFROM v_orders\n\n'

In [2]:
#Assignment Question 2
'''

Using Analytical Functions find the difference between the order total for each order_id and the order_id with the 
maximum order total for that month/year. 
(For the assignment questions we are using V_ORDERS created in the "Assignment Data Preparation" lecture)

select order_id , TO_CHAR(ORDER_DATETIME, 'MM-YY') AS MONTH_YEAR FROM v_orders;
SELECT
    ORDER_ID,
    TO_CHAR(ORDER_DATETIME,'MM-YY') AS MONTH_YEAR,
    ORDER_TOTAL,
    ORDER_TOTAL - MAX(ORDER_TOTAL) OVER(PARTITION BY TO_CHAR(ORDER_DATETIME,'MM-YY')) AS DIFF_FROM_MAX
FROM
V_ORDERS;
'''

'\n\nUsing Analytical Functions find the difference between the order total for each order_id and the order_id with the \nmaximum order total for that month/year. \n(For the assignment questions we are using V_ORDERS created in the "Assignment Data Preparation" lecture)\n\nselect order_id , TO_CHAR(ORDER_DATETIME, \'MM-YY\') AS MONTH_YEAR FROM v_orders;\nSELECT\n    ORDER_ID,\n    TO_CHAR(ORDER_DATETIME,\'MM-YY\') AS MONTH_YEAR,\n    ORDER_TOTAL,\n    ORDER_TOTAL - MAX(ORDER_TOTAL) OVER(PARTITION BY TO_CHAR(ORDER_DATETIME,\'MM-YY\')) AS DIFF_FROM_MAX\nFROM\nV_ORDERS;\n'

In [2]:
#Assignment Question 3
'''
Using Analytical Functions find the order total for each order_id and subtract the 3 month rolling average order total (the average of the current month and the previous 2 months of orders).
Your solution should only calculate the rolling average for months that are in the same year.

(For the assignment questions we are using V_ORDERS created in the "Assignment Data Preparation" lecture)


select 
  order_datetime,
  to_char(order_datetime, 'MMYY') as MONTH_YEAR1,
  to_number(to_char(order_datetime, 'MMYY') ) as MONTH_YEAR2,
    to_number(to_char(order_datetime, 'MMYYYYY') ) as MONTH_YEAR3
from v_orders;


# Note 
You may be wondering why I haven't partitioned the window function by Year in my solution for Question 3.
This is because it's not required. I am defining my window range as 2 preceding, so when the year is 202101, 2 preceding from that 
is 202101-2 = 202099, so the previous years months are excluded from the window frame and it's naturally partitioned by the year.


select 
  order_id,
  to_char(order_datetime, 'MM-YY') as MONTH_YEAR,
  order_total,
  AVG(order_total) OVER(order by TO_NUMBER(TO_CHAR(ORDER_DATETIME, 'YYMM')) RANGE 2 PRECEDING) as AVG_3_MONTH,
  ORDER_TOTAL - AVG(order_total) OVER(order by TO_NUMBER(TO_CHAR(ORDER_DATETIME, 'YYMM')) RANGE 2 PRECEDING) as VARIANCE

from v_orders;

'''

'\nUsing Analytical Functions find the order total for each order_id and subtract the 3 month rolling average order total (the average of the current month and the previous 2 months of orders).\nYour solution should only calculate the rolling average for months that are in the same year.\n\n(For the assignment questions we are using V_ORDERS created in the "Assignment Data Preparation" lecture)\n\n\nselect \n  order_datetime,\n  to_char(order_datetime, \'MMYY\') as MONTH_YEAR1,\n  to_number(to_char(order_datetime, \'MMYY\') ) as MONTH_YEAR2,\n    to_number(to_char(order_datetime, \'MMYYYYY\') ) as MONTH_YEAR3\nfrom v_orders;\n\n\n# Note \nYou may be wondering why I haven\'t partitioned the window function by Year in my solution for Question 3.\nThis is because it\'s not required. I am defining my window range as 2 preceding, so when the year is 202101, 2 preceding from that \nis 202101-2 = 202099, so the previous years months are excluded from the window frame and it\'s naturally parti

In [3]:
# BONUS (Running Totals): Assignment Question 4
'''

Create the following view:

CREATE VIEW V_ORDER_YEAR_MONTH AS
(SELECT TO_CHAR(ORDER_DATETIME, 'YYYY-MM') AS YEAR_MONTH,
SUM(ORDER_TOTAL) AS YEAR_MONTH_TOTAL
FROM V_ORDERS
GROUP BY TO_CHAR(ORDER_DATETIME,'YYYY-MM'));


On the above view use Analytical Functions to create a running total column using the YEAR_MONTH_TOTAL field ordered  by the YEAR_MONTH field in ascending order
'''


"\n\nCreate the following view:\n\nCREATE VIEW V_ORDER_YEAR_MONTH AS\n(SELECT TO_CHAR(ORDER_DATETIME, 'YYYY-MM') AS YEAR_MONTH,\nSUM(ORDER_TOTAL) AS YEAR_MONTH_TOTAL\nFROM V_ORDERS\nGROUP BY TO_CHAR(ORDER_DATETIME,'YYYY-MM'));\n\n\nOn the above view use Analytical Functions to create a running total column using the YEAR_MONTH_TOTAL field ordered  by the YEAR_MONTH field in ascending order\n"

In [ ]:
# BONUS (Running Totals): Assignment Question 4 Solution (SQL Code)
'''
SELECT

YEAR_MONTH,

YEAR_MONTH_TOTAL,

SUM(YEAR_MONTH_TOTAL) OVER(ORDER BY YEAR_MONTH ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)

FROM V_ORDER_YEAR_MONTH;

OR

SELECT

YEAR_MONTH,

YEAR_MONTH_TOTAL,

SUM(YEAR_MONTH_TOTAL) OVER(ORDER BY YEAR_MONTH ASC RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)

FROM V_ORDER_YEAR_MONTH;
'''